In [1]:
using StaticArrays, ForwardDiff, BenchmarkTools

function jacob_TM(u, p, t)
    
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    U_y(y) = (50.0 * ΔU0 * exp(-50.0 * (y - ythr))) / (1+exp(-50.0*(y - ythr)))^2
    
    E, x, y = u
    
    E_E = ( -1 + ((J * U(y) * x * exp((J * U(y) * x * E + I0) / α))) / (1 + exp((J * U(y) * x * E + I0)/α)) ) / τ
    E_x = (J * U(y) * E * exp((J * U(y) * x * E + I0) / α)) / (τ * (1 + exp((J * U(y) * x * E + I0) / α) ))
    E_y = ( J * U_y(y) * x * E * exp((J * U(y) * x * E + I0) / α) ) / (τ * (1 + exp((J * U(y) * x * E + I0) / α)) )
    
    x_E = -U(y)*x
    x_x = -1/τD - U(y)*E
    x_y = -U_y(y)*x*E
    
    y_E = 0
    y_x = 20.0 * β * exp( (-20.0) * (x - xthr) ) / (1 + exp( (-20.0) * (x-xthr) ))
    y_y = -1/τy
    
    SMatrix{3,3}(E_E, x_E,y_E,
                E_x, x_x, y_x,
                E_y, x_y, y_y)
end

In [2]:
function jacob_TM_(u, p, t)
    
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x - p[6]) ) )
    U_y(y) = ( 50.0 * p[9] * exp(-50.0 * (y - p[7])) ) / (1.0 + exp( -50.0*(y - p[7]) ) )^2
    g(E, x, y) = exp((p[5]  * U(y) * x * E + p[11]) / p[1])
    σ_der(x) = exp( (-20.0) * (x - p[6]) )
    
    E_E = ( -1.0 + ((J * U(u[3]) * u[2] * g(u[1], u[2], u[3]))) / (1.0 + g(u[1], u[2], u[3])) ) / p[2]
    E_x = (p[5] * U(u[3]) * u[1] * g(u[1], u[2], u[3])) / (p[2] * (1.0 + g(u[1], u[2], u[3]) ))
    E_y = ( p[5]  * U_y(u[3]) * u[2] * u[1] * g(u[1], u[2], u[3]) ) / (p[2] * (1.0 + g(u[1], u[2], u[3])) )
    
    x_E = -U(u[3])*u[2]
    x_x = -1.0 / p[3] - U(u[3])*u[1]
    x_y = (-U_y(u[3])) * u[2] * u[1]
    
    y_x = 20.0 * p[10] * σ_der(u[2]) / (1.0 + σ_der(u[2]))^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end

jacob_TM_ (generic function with 1 method)

In [3]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; const U0 = 0.3;
I0 = -1.6;

In [4]:
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = SA[8.124338914021985, 0.6552902588595779, 0.4438414263930182]
t = 0;

In [5]:
function TM(u, p, t)
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end

TM (generic function with 1 method)

In [6]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> TM(x, p, t), u0)

Jacobi (generic function with 1 method)

In [7]:
@benchmark jacob_TM_(u0, p, t)

BenchmarkTools.Trial: 10000 samples with 681 evaluations.
 Range (min … max):  191.630 ns …   8.210 μs  ┊ GC (min … max): 0.00% … 97.25%
 Time  (median):     203.524 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   217.990 ns ± 124.235 ns  ┊ GC (mean ± σ):  1.70% ±  3.21%

  ▃ ▅▂█▅▂▁  ▃▃▁▁    ▁▁                                     ▂  ▂ ▁
  ██████████████▇██████████▇▇▇▇▇▇██▇▆▇▇▅▄▅▅▅▅▄▁▄▃▃▄▄▃▄▄▃▆▃▃██▆█ █
  192 ns        Histogram: log(frequency) by time        345 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

In [8]:
@benchmark Jacobi(u0)

BenchmarkTools.Trial: 10000 samples with 294 evaluations.
 Range (min … max):  264.966 ns …  12.707 μs  ┊ GC (min … max): 0.00% … 97.41%
 Time  (median):     281.633 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   304.740 ns ± 428.993 ns  ┊ GC (mean ± σ):  6.22% ±  4.32%

     ▁  █▂                                                       
  ▄▅▃█▆▄██▄▆▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  265 ns           Histogram: frequency by time          417 ns <

 Memory estimate: 336 bytes, allocs estimate: 4.